In [1]:
from willireply.data import enron
from willireply.features import features
from willireply.features.feature_extractor import FeatureExtractor

/Users/jamwheel/anaconda3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, fbeta_score

from pathlib import Path
from tqdm.autonotebook import tqdm
from sklearn.linear_model import SGDRegressor
import numpy as np
import ipywidgets as widgets

# Standard Test

In [10]:
df_train = enron.get_dataframe('skilling-j', received_only=True)
np.log(1+features.words_in_body(df_train))

id
1       4.204693
2       5.700444
3       5.493061
4       3.761200
5       5.342334
6       1.791759
7       4.969813
8       5.707110
9       5.023881
10      2.484907
11      4.025352
12      7.254178
13      5.293305
14      5.105945
15      4.330733
16      2.564949
17      4.060443
18      6.066108
19      2.484907
20      2.890372
21      4.304065
22      6.246107
23      4.744932
24      5.771441
25      2.639057
26      4.158883
27      3.737670
28      4.262680
29      5.231109
30      4.317488
          ...   
4096    6.194405
4097    4.025352
4098    3.367296
4099    6.489205
4100    6.831954
4101    7.063904
4102    5.752573
4103    3.784190
4104    5.147494
4105    3.295837
4106    4.875197
4107    3.663562
4108    4.770685
4109    3.465736
4110    6.570883
4111    4.955827
4112    5.968708
4113    4.317488
4114    4.304065
4115    5.991465
4116    5.017280
4117    3.044522
4118    3.367296
4119    4.276666
4120    3.401197
4121    4.875197
4122    4.969813
4123    4.4

In [19]:
from willireply.features import features

my_common_words = ['ASAP', 'please', 'could you']
subject_common_words_feature = lambda df: features.common_words_subject(df, ['?' '!'])
body_common_words_feature = lambda df: features.common_words_body(df, ['?', '!'])

fe = FeatureExtractor(
      subject_common_words_feature,
      body_common_words_feature,
      #features.thread_length,
      lambda df: np.log(1+features.words_in_body(df)),
      features.words_in_subject
                     )

In [5]:
#users = [f.stem for f in enron.ENRON_INDEX_FOLDER.iterdir() if enron.is_labeled(f.stem)]

In [20]:
users = ['skilling-j']

In [32]:
model = SGDRegressor(max_iter=100)

In [41]:
for user in tqdm(users[:20]):
    df_train = enron.get_dataframe(user, received_only=True)
    if len(df_train) == 0:
        print(user,"has no emails")
        continue
    X = fe.extract(df_train)
    y_true = fe.get_labels(df_train)
    if len(np.where(y_true)[0]) == 0:
        print(user,"has no replies")
        continue

    model.fit(X, y_true)

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

In [37]:
#df_train = enron.get_dataframe('may-l', received_only=True)
df_validate = enron.get_dataframe('skilling-j', received_only=True)

In [42]:
y_pred = model.predict(fe.extract(df_validate)) > 0# Not sure best place to set this predict...
y_true = fe.get_labels(df_validate)

print(classification_report(y_true, y_pred, target_names=["no reply", "reply"]))
print('f_2 = %s' % fbeta_score(y_true, y_pred, 2, labels=['no reply', 'reply'], pos_label=1))

             precision    recall  f1-score   support

   no reply       0.97      0.34      0.50      3411
      reply       0.03      0.66      0.06       109

avg / total       0.94      0.35      0.49      3520

f_2 = 0.1305293691080493


In [27]:
df_all = enron.get_dataframe(users[0])

In [28]:
df_all.loc[61].body.split('-----Original Message-----')[0]

'Confirmed for 10:30am on Wed Dec 13.  Your place our ours?\n\nThanks Sherri.\n\n'

In [29]:
df2 = df_all.copy()
df2['body'] = df_all['body'].apply(lambda x: x.split('-----Original Message-----')[0])
print(df2.loc[61].body)

Confirmed for 10:30am on Wed Dec 13.  Your place our ours?

Thanks Sherri.




In [30]:
missed = df_validate.iloc[np.where(y_true > y_pred)[0]]

labels = []
left = []
right = []

for idx, row in missed.iterrows():
    left.append(row.body)
    labels.append(f'({idx}) {row.subject}')
    right.append(df_all.loc[row.reply_id].body)
    
accordion = widgets.Accordion(children=[
    widgets.Box( [widgets.Textarea(l, layout={'height': '500px'}), widgets.Textarea(r, layout={'height': '500px'})] )
    for (l, r) in zip(left, right)
])
for i, l in enumerate(labels):
    accordion.set_title(i, l)
accordion

Accordion(children=(Box(children=(Textarea(value="Jeff,\n\nGreat to see Enron and you doing so well.  Congratulations on the -belated-\nCEO ship, the blowout first quarter,\nand the recent piece in Bloomberg.\n\nSince we last met Luminant has done a important and impactful work for Enron\n(EES, Enron Networks, Global\nSourcing, Broadband among others).  The Oasis and the Websource Intranet\nsites are examples of the results of\nthis work.  Phillippe recently selected Luminant as one five integrators\nthat Enron will use going forward.\n\nDavid Quackenbush, Luminants COO, has been personally involved in leading\nthe work at Enron.  He and\nJim Corey Luminant's CEO would like to have the opportunity of meeting with\nyou, bringing you up to date\non the work, and inviting you to consider joining the Luminant board.\n\nI am no longer affiliated with Luminant but David asked me to let you know\nof their interest.  He will be\nfollowing up and seeing if there will be an opportunity for you to meet with\nthem.\n\nRegards,\n\nGil Marmol\n\n\n", layout=Layout(height='500px')), Textarea(value='Gil,\n\nJeff asked me to let you know that he really appreciates this opportunity, but his time in the office is at such a premium right now due to his heavy travel schedule, it would be difficult at best to work them in.  He has turned down several invitations to join boards because he just can\'t commit the time.  So, while he is flattered by their interest, if and when Mr. Quackenbush calls we will decline.\n\nIt was nice talking to you.  Glad to hear all is well.\n\nYours faithfully, Sherri ;-)\n\n\n\n\n"Gil Marmol" <gmarmol@gmarmol.com> ', layout=Layout(height='500px')))), Box(children=(Textarea(value='NEW CALL-IN NUMBERS:\nNOTE:  This week there will be an additional  call on Friday 12/15  at 8:00am \nHouston Time\nDabhol Task Force Conference Call - Every Monday\n8:00 am - Houston Time\nDomestic 1-888-689-5736\nInternational: 1-847-944-7277\nPasscode: 6530874#\nHost: Wade Cline', layout=Layout(height='500px')), Textarea(value='Jana, would you please add me to your cc: list for these calls.  Thanks.\n\nRegards,\nSherri Sera\nAssistant to Jeff Skilling\n713.853.5984\n713.646.8381\nsherri.sera@enron.com\n\n\nTo: Kenneth Lay@ENRON, Jeff Skilling@ENRON, Rebecca \nMcDonald/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, James A \nHughes/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Rob Walls/NA/Enron@Enron, Bruce \nLundstrom/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, John \nAmbler/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Wade \nCline/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Neil \nMcGregor/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Mohan \nGurunath/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Ananda \nMukerji/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Bill \nGathmann/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Sandeep \nKohli/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Paul \nKraske/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Sandeep \nKatwala/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, T \nRavishanker/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Jimmy \nMogal/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, P \nMohanram/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT\ncc: Loretta Brelsford/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Connie \nBlackwood/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Debra Hicks/NA/Enron@Enron, \nWendi Hoelscher/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Sandy \nHoelscher/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Mrudula \nGadade/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT \n\nSubject: Dabhol Task Force Weekly Call\n\nNEW CALL-IN NUMBERS:\nNOTE:  This week there will be an additional  call on Friday 12/15  at 8:00am \nHouston Time\nDabhol Task Force Conference Call - Every Monday\n8:00 am - Houston Time\nDomestic 1-888-689-5736\nInternational: 1-847-944-7277\nPasscode: 6530874#\nHost: Wade Cline\n\n\n', layout=Layout(height='500px')))), Box(children=(Textarea(value="We are in urgent need of approximately 35 interviewers for this weekend's \nSuper Saturday recruiting, to which 105 analyst and associate candidates have \nbeen invited.  You all realize the importan